**tensorflow**



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

In [70]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

In [ ]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000,10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200,10))

model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))
model.summary()

In [ ]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

**pytorch**

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torchmetrics
print(torch.__version__)

In [76]:
net = nn.Sequential(nn.Linear(72, 32), nn.ReLU(), nn.Linear(32, 32), nn.ReLU(), nn.Linear(32, 10)) 

def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)  # 1维的包含n个（n为样本总数）分量的tensor
    cmp = y_hat.type(y.dtype) == y  # 1维的包含n个分量的tensor，并且每个分量的值都为True或者False
    return float(cmp.type(y.dtype).sum())  # 在这里实际上需要将True和False转成对应的1和0，因此，我们执行cmp.type(y.dtype)，将True或者false转成1或者0

def data_iter(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples))
  random.shuffle(indices)
  for i in range(0, num_examples, batch_size):
    batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
    yield features[batch_indices], labels[batch_indices]

In [ ]:
batch_size = 100

loss = nn.CrossEntropyLoss()

optim =  torch.optim.Adam(net.parameters(), lr=0.001)

# 准备数据
train_x = torch.rand((1000, 72), requires_grad=True)
train_y = torch.empty(1000, dtype=torch.long).random_(10)

val_x = torch.rand((200, 72))
val_y = torch.empty(200, dtype=torch.long).random_(10)

num_epochs = 10
metric = torchmetrics.Accuracy()
for epoch in range(num_epochs):
  for X, y in data_iter(batch_size, train_x, train_y): 
    y_hat = net(X)
    l = loss(y_hat, y)
    optim.zero_grad()
    l.backward()
    # print(net[0].weight.grad)
    optim.step()
    acc = metric(y_hat.softmax(dim = -1), y)
  print(f"loss {l.detach().item()}, accracy {acc.detach().item()}. ")

In [ ]:
test_x = torch.rand((1000, 72))
test_y = torch.empty(1000, dtype=torch.long).random_(10)

num_epochs = 10
for epoch in range(num_epochs):
  test_y_hat = net(test_x)
  acc = metric(test_y_hat.softmax(dim = -1), test_y)
  print(f"accracy {acc.detach().item()}. ")